In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
from os.path import join as p_join
import sys
from typing import List, Tuple, Dict, Set, Any, Optional, Callable
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent))
import requests
from tqdm import tqdm
try:
    from fabulous import color as fb_color
    color_print = lambda x, color='green': print(getattr(fb_color, color)(x)) if 'fb_color' in globals() else print(x)
except Exception as e:
    color_print = lambda x, color='green': print(x)

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
from src.parse_utils import get_events_list, get_events_info, get_fighters_info, get_one_fight_stats
from src.processing import eventslist2df

In [7]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

---

In [9]:
PARSED_DATA_PATH = p_join(str(Path(os.getcwd()).parent), 'data/raw/all_fights.json')
print(f"PARSED_DATA_PATH: {PARSED_DATA_PATH}")

PARSED_DATA_PATH: /home/aiandrejcev/ufc/data/raw/all_fights.json


In [10]:
fights_list, status_ok = get_events_list()
print(status_ok)
print(len(fights_list))
fights_list[:3]

True
630


[{'event_url': 'http://www.ufcstats.com/event-details/56ec58954158966a',
  'event_name': 'UFC Fight Night: Cannonier vs. Strickland',
  'date': 'December 17, 2022',
  'location': 'Las Vegas, Nevada, USA'},
 {'event_url': 'http://www.ufcstats.com/event-details/f65a0eb902f9476b',
  'event_name': 'UFC 282: Blachowicz vs. Ankalaev',
  'date': 'December 10, 2022',
  'location': 'Las Vegas, Nevada, USA'},
 {'event_url': 'http://www.ufcstats.com/event-details/b23388ff8ac6637b',
  'event_name': 'UFC Fight Night: Thompson vs. Holland',
  'date': 'December 03, 2022',
  'location': 'Orlando, Florida, USA'}]

In [13]:
fights_df = eventslist2df(fights_list)
fights_df

,event_url,event_name,date,location
0,http://www.ufcstats.com/event-details/56ec5895...,UFC Fight Night: Cannonier vs. Strickland,"December 17, 2022","Las Vegas, Nevada, USA"
1,http://www.ufcstats.com/event-details/f65a0eb9...,UFC 282: Blachowicz vs. Ankalaev,"December 10, 2022","Las Vegas, Nevada, USA"
2,http://www.ufcstats.com/event-details/b23388ff...,UFC Fight Night: Thompson vs. Holland,"December 03, 2022","Orlando, Florida, USA"
3,http://www.ufcstats.com/event-details/012fc7cd...,UFC Fight Night: Nzechukwu vs. Cutelaba,"November 19, 2022","Las Vegas, Nevada, USA"
4,http://www.ufcstats.com/event-details/b3b6e80b...,UFC 281: Adesanya vs. Pereira,"November 12, 2022","New York City, New York, USA"
...,...,...,...,...
625,http://www.ufcstats.com/event-details/1c3f5e85...,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, USA"
626,http://www.ufcstats.com/event-details/dedc3bb4...,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, USA"
627,http://www.ufcstats.com/event-details/b60391da...,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, USA"
628,http://www.ufcstats.com/event-details/1a49e067...,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, USA"


In [8]:
# from src.parse_utils import get_events_info

In [9]:
# event_urls = ['http://www.ufcstats.com/event-details/a23e63184c65f5b8']
# events_stats_dict = {}
# for event_url in tqdm(event_urls):
#     one_event = requests.get(event_url)
#     one_event = BeautifulSoup(one_event.content, 'lxml')

#     fights_in_1_event = (
#             one_event
#             .find_all(
#                     "tr", {
#                             "class": "b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click"
#                     }
#             )
#     )

#     stat_cols_names = (
#             one_event.find_all('thead', {'class': "b-fight-details__table-head"})[0]
#             .find_all('th', {'class': "b-fight-details__table-col"})
#     )
#     item_number2stat_mapping = {
#             i: stat.text.strip().lower().replace('/', '_').replace(' ', '_') for i, stat in enumerate(stat_cols_names)
#     }

#     one_event_stats_list = []
#     for i, one_fight in enumerate(fights_in_1_event):
#         one_fights_stats = one_fight.find_all('td', {"class": "b-fight-details__table-col"})

#         curr_fight_stats_dict = {}
#         for j, stat in enumerate(one_fights_stats):
#                 stat = stat.find_all('p', {'class': 'b-fight-details__table-text'})
#                 if len(stat) == 1:
#                         curr_fight_stats_dict[item_number2stat_mapping[j]] = stat[0].text.strip()
#                 elif len(stat) == 2:
#                         curr_fight_stats_dict[item_number2stat_mapping[j]] = []
#                         for val in stat:
#                                 curr_fight_stats_dict[item_number2stat_mapping[j]].append(val.text.strip())
#                 else:
#                         raise ValueError("it's not expected to be more than 2 values!")

#         one_event_stats_list.append(curr_fight_stats_dict)
#     events_stats_dict[event_url] = one_event_stats_list

In [10]:
# fights_info_dict = get_events_info(event_urls=fights_df['event_url'].values.tolist())
# print(len(fights_info_dict))

In [9]:
all_fights_list = []
for event_uri, event_date, event_location, event_name in tqdm(
    fights_df[['event_url', 'date', 'location', 'event_name']].itertuples(index=False),
    total=len(fights_df)
):
    one_event = requests.get(event_uri)
    one_event = BeautifulSoup(one_event.content, 'lxml')
    one_event = (
        one_event
        .find_all(
            "tr", {"class": "b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click"}
        )
    )
    for one_fight in one_event:
        fight_uri = [
            row.replace('data-link', '').replace('=', '').replace('"', '') 
            for row in str(one_fight).split() if 'data-link' in row
        ][0]

        fight_stats_dict = get_one_fight_stats(fight_uri=fight_uri)
        fight_stats_dict['date'] = event_date
        fight_stats_dict['location'] = event_location
        fight_stats_dict['event_name'] = event_name
        fight_stats_dict['event_uri'] = event_uri
        fight_stats_dict['fight_uri'] = fight_uri
        all_fights_list.append(fight_stats_dict)


  7%|█████████▎                                                                                                                      | 46/630 [09:00<2:03:22, 12.68s/it]


 15%|██████████████████▉                                                                                                             | 93/630 [17:39<1:30:01, 10.06s/it]


 22%|████████████████████████████▏                                                                                                  | 140/630 [26:36<1:40:44, 12.34s/it]


 30%|█████████████████████████████████████▋                                                                                         | 187/630 [36:05<1:32:00, 12.46s/it]


 37%|███████████████████████████████████████████████▏                                                                               | 234/630 [45:06<1:17:30, 11.74s/it]


...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 627/630 [1:50:28<00:20,  6.96s/it]

can't get per round statistics!
list index out of range




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 628/630 [1:50:35<00:13,  6.85s/it]

can't get per round statistics!
list index out of range




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 630/630 [1:50:50<00:00, 10.56s/it]


In [11]:
json.dump(
    all_fights_list, 
    open(PARSED_DATA_PATH, mode='w', encoding='utf-8'), ensure_ascii=False, indent=2
)

NameError: name 'json' is not defined

In [10]:
from src.parse_utils import get_fighters_info

In [11]:
overall_fighters_list, status_ok = get_fighters_info()
print(f"status_ok: {status_ok}")
print(len(overall_fighters_list))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:30<00:00,  1.18s/it]

status_ok: True
476


In [400]:
# are there namesakes among fighters?
res = Counter([f"{dct['First']}_{dct['Last']}" for dct in overall_fighters_list])
res.most_common(1)

[('Tom_Aaron', 1)]

In [ ]:
n = 2
unq = {1, 2}
p(n) = 1/2

In [ ]:
"""
                        m
         1         2        3         4          5
    |--------||--------||--------||--------||--------|
    |        ||        ||        ||        |         |
  1 |    1   ||   0    ||    0   ||    0   |     0   | 
    |        ||        ||        ||        |         |
    |--------||--------||--------||--------||--------|
    |        ||        ||        ||        |         |
  2 |    0   ||   1/2  ||(1/2)^2*||(1/2)^3*|(1/2)^4* | 
    |        ||        ||(1/2)   ||(1/2)   |(1/2)    |
    |--------||--------||--------||--------||--------|
    |        ||        ||        ||        |         |
  3 |    0   ||   0    ||3!/3^3  ||p(3, 2)*|        |
    |        ||        ||        ||(1/n)   |         | 
    |--------||--------||--------||--------||--------| 
    |        ||        ||        ||        |         |
  4 |    0    ||        ||        ||        |        | 
    |        ||        ||        ||        |         | 
    |--------||--------||--------||--------||--------|
    |        ||        ||        ||        |         |
  5 |    0    ||        ||        ||        |        |
    |        ||        ||        ||        |         |
    |--------||--------||--------||--------||--------|
    

"""

In [ ]:
p(4, 3) = p(3, 2)*(1/n) + p(2, 1)*p(2, 2) + p(2, 2)*p(2, 1)

In [ ]:
p(m, n) = ???
p(1, 1) = 1
p(2, 2) = 1/2
p(3, 2) = 
                      m
       \   1  2  3  4  5  6  7 ...
    1      1 
    
n   2
    
    3
    
    4
    
    5

In [ ]:
n = 2
p(m=2) = 1/2
p(m=3) = (1/2)^2 * (1/2)
p(m=4) = (1/2)^3 * (1/2)
...
p(m=x) = (1/2)^(x-1) * (1/2)


n=3 
p(m=1,2) = 0
p(m=3) = 3!/3^3
p(m=4) = (1 - p(m=3))*

In [ ]:
p(3) = p(2)*(1/n) + p(1)

In [ ]:
Omega_(m=2) = {(1, 1), (1, 2), (2, 1), (2, 2)}
Omega_(m=3) = {
    (1, 1, 1), (1, 1, 2), (1, 2, 1), (1, 2, 2), (2, 1, 1), (2, 1, 2), (2, 2, 1), (2, 2, 2)
}
Omega_(n=3) = {
    (1, 1, 1), 
    (1, 1, 2), (1, 2, 1), (1, 2, 2), (2, 1, 1), (2, 1, 2), (2, 2, 1), (2, 2, 2), 
    (1, 1, 3), (1, 3, 1), (1, 3, 3), (3, 1, 1), (3, 1, 3), (3, 3, 1), (3, 3, 3),
    (2, 2, 2), 
    (2, 2, 3), (2, 3, 2), (2, 3, 3), (3, 2, 2), (3, 2, 3), (3, 3, 2)
}